# Sum & product types, and railways

First you'll practice working with enums and then you can expand the railway example from the lecture.

Have fun!

### Exercise 1

Write the following functions that return options, as there is not always a return value. Don't use any loops, always write them recursively with pattern matching.

1. The function `firstItem` that returns the first item from a list.
2. The function `lastItem` that returns the last item from a list.
3. The function `sumTwoOptions` that sums two options of numbers, there is only a result when both arguments are `Some`.
4. The function `sum` that computes the sum of a list of integers (can be tail recursive).
5. The function `max` that returns the largest item in a list of integers (can be tail recursive).
6. The function `find` that returns the first element that matches. (signature: `('T -> bool) -> 'T list -> 'T Option`)

In [17]:
let firstItem = fun ls ->
    match ls with
        | [] -> None
        | head :: _ -> Some head

let rec lastItem = fun ls ->
    match ls with
        | [] -> None
        | head :: [] -> Some head 
        | head :: tail -> lastItem tail

let rec sumTwoOptions = fun x y ->
    match (x, y) with
        | (Some x, Some y) -> Some (x + y)
        | _ -> None



### Exercise 2

Another good use of enums (discriminated unions) is to encode possible states of information. In this exercise you will keep track of contact information. We want to be able to store phone numbers and email addresses. Not everyone needs to have a phone number or an email address, so all four combinations are allowed (neither, only phone, only email, both). 

1. Write one enum (discriminated union) type `ContactInfo` that can store all four states. Note that one does not contain a field, two contain a string and one contains a tuple of two strings.
2. Now we want to add a phone number. Write a function `updatePhone: string -> ContactInfo -> ContactInfo` that adds the phone number to the given ContactInfo instance. It should handle all cases!
3. Do the same for `updateEmail`.
4. Now write a function `sendEmail` that mocks sending an email. This function should only send the email if an email address is known. You can mock sending an email by just printing "sent msg to me@example.com" in the console.
5. Test your implementation

Congrats, you just implementated a simple state pattern!

The enums are very handy to encode the state you are currently in. Then all the state changing functions are simple match functions that return an updated enum.

You can think about how you would use this to for example implement a state pattern for a webshop. The state of the cart could be empty, filled but not paid, and then paid. The mutating functions could add items to the cart or could process payment.

### Exercise 3

In the lecture we implemented the `connect` function that makes railway oriented error handling possible. You find this function below, together with mocked versions of `validate`, `updateDb`, and `sendEmail`. You'll expand on this example in two ways:

1. We want to add a prefix to each `request`. This function should take a string `s` and return `"Request" + s`. This function will never produce an error, so it cannot switch to the error track! Think about whether we can still apply `connect` to it and then add the prefix step between `validate` and `updateDb`.
2. Our use case slightly changed, we now always want to send an email, to keep everyone updated even if an error occurs in validation or in the database. However, sending the email can still go wrong. At the end of track we still need a `Result` (`Ok` if the email was sent successfully, `Error` if the email failed). We should never get an `Error` from the database, as we just keep everyone up to date with the email and then it's fine. 
In order to do this we need to be able to switch back from the error track to the happy track. Take a look at the last slide of lecture 4 for the complete track we want to build.
Now write the function `recover` that simply prepends the string with whether the request failed or not. When recover receives `Ok value` it returns the string `"success: value"`, when it receives `Error value` it returns the string `"error: value"`.
Then write the function `catch` to connect `recover` in the pipeline and test the pipeline!

In [ ]:
let rnd = (new System.Random())

let validate (request: string) =
    if request.Length > 5 then Ok request
    else Error "invalid request"

let updateDb request =
    let prob = rnd.NextDouble()
    if prob < 0.1 then Error "database unavailable"
    else if prob < 0.3 then Error "request unknown in database"
    else Ok request

let sendEmail request = 
    let prob = rnd.NextDouble()
    if prob < 0.1 then Error "sending email failed"
    else Ok ("email sent successfully (" + request + ")")

let connect func request = 
    match request with
        | Ok value -> func value
        | Error value -> Error value

let handleUseCase = validate >> connect updateDb >> connect sendEmail

handleUseCase "validRequest"

When you're all done with this exercise you should think back to JS promises for a moment. You can chain promises together with `then` and recover from errors with `catch`. You might have not used it much, but you can keep going with the chain after a `catch`. 

JS promise chaining is Railway Oriented Programming! 😍

(btw.. I can't get this to work with promises in the notebook, you might have to run it elsewhere if you want to run it.)

In [ ]:
const chain = new Promise((resolve, _) => resolve("value"))

chain
    .then(val => new Promise((_, reject) => reject("oh nee toch niet")))
    .catch(err => new Promise((resolve, _) => resolve("weer opgelost")))
    .then(console.log)